In [19]:
import requests
from lxml import etree
import pandas as pd
import time
import random

data_id = []
data_star = []
data_date = []
data_comment = []

In [20]:
def scraping_page(start, l1, l2, l3, l4, code = "35465232"):
    url = f"https://movie.douban.com/subject/{code}/comments?start={start}"
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36"
        }
    
    try:
        response = requests.get(url, headers = headers)
        response.raise_for_status()
        time.sleep(random.randint(5,10))

        parser = etree.HTMLParser() # 创建HTML解析器对象，默认配置
        tree = etree.fromstring(response.text, parser) # 使用上面的解析器解析网页内容
        # 也可以用tree = etree.HTML(response.text)，默认使用HTMLParser解析
        divs = tree.xpath('//div[@class = "comment"]')
        for div in divs:
            id = div.xpath('./h3/span[@class = "comment-info"]/a/text()')[0]
            star = div.xpath('./h3/span[@class = "comment-info"]/span')[1].get("class")[7]
            date = div.xpath('./h3//span[@class = "comment-time "]/text()')[0].strip()
            comment = div.xpath('.//span[@class = "short"]/text()')[0]
            l1.append(id)
            l2.append(star)
            l3.append(date)
            l4.append(comment)
        print(f"{start + 1} to {start + 20} done!")

    except Exception as e:
        print(f"Failed! {e}")

In [ ]:
for start in range(0, 100, 20):
    scraping_page(start, data_id, data_star, data_date, data_comment)

In [24]:
if __name__ == '__main__':
    data = {
        '用户名': data_id,
        '评分': data_star,
        '日期': data_date,
        '评论': data_comment
    }
    df = pd.DataFrame(data)
    df.to_excel('douban_comments.xlsx', index=False)